# GoFish Graphics Python - Test Notebook

This notebook tests the `gofish-graphics` Python wrapper.

**Note**: JavaScript is the source of truth for all rendering. This Python package provides a syntax-similar API that translates Python calls to JavaScript execution.

## Kernel

This notebook uses the **`gofish-python`** kernel (Python with gofish-graphics environment).

If you don't see it in the kernel selector:
1. Press `Cmd+Shift+P` (Mac) or `Ctrl+Shift+P` (Windows/Linux)
2. Type "Select Kernel" and choose "Python (gofish-python)"
3. Or use the kernel selector in the notebook toolbar

## Prerequisites

1. ✅ Package installed in development mode
2. ✅ JavaScript bridge installed (pythonmonkey)
3. ⚠️ GoFish JavaScript bundle (needs to be built if rendering is needed)


In [4]:
# Import the package
try:
    from gofish_python import chart, spread, stack, rect, circle, line, area, group, scatter
    from gofish_python import display_jupyter, display_html
    print("✓ GoFish Graphics imported successfully")
except ImportError as e:
    print(f"✗ Import failed: {e}")
    print("\nMake sure you've installed the package:")
    print("  uv sync  # or: pip install -e .")


✓ GoFish Graphics imported successfully


In [2]:
# Test JavaScript bridge
try:
    from gofish_python.js_bridge import get_js_bridge
    bridge = get_js_bridge()
    print(f"✓ JavaScript bridge initialized: {bridge._initialized}")
    
    # Check if bundle was found
    if hasattr(bridge, '_gofish_dist') and bridge._gofish_dist:
        print(f"✓ GoFish bundle found at: {bridge._gofish_dist}")
        
        # Try to load the module
        try:
            gofish_js = bridge.gofish
            print("✓ GoFish JavaScript module loaded successfully")
        except Exception as e:
            print(f"⚠ GoFish JS module loading issue: {e}")
            print("\nNote: ES module loading may require additional setup.")
            print("The Python API structure is available, but full rendering")
            print("may need proper ES module support.")
    else:
        print("⚠ GoFish bundle not found")
        print("Build it with: cd packages/gofish-graphics && pnpm build")
        print("Or set GOFISH_JS_PATH environment variable")
except Exception as e:
    print(f"✗ Bridge test failed: {e}")
    print("\nMake sure you've installed a JavaScript bridge:")
    print("  uv pip install pythonmonkey")


✓ JavaScript bridge initialized: True
✓ GoFish bundle found at: /Users/jmp/gofish-20250316/packages/gofish-graphics/dist
✓ GoFish JavaScript module loaded successfully


## Example 1: Simple Bar Chart

A basic bar chart showing letter frequencies.


In [3]:
# Sample data
alphabet = [
    {"letter": "A", "frequency": 28},
    {"letter": "B", "frequency": 55},
    {"letter": "C", "frequency": 43},
    {"letter": "D", "frequency": 91},
    {"letter": "E", "frequency": 81},
    {"letter": "F", "frequency": 53},
    {"letter": "G", "frequency": 19},
    {"letter": "H", "frequency": 87},
    {"letter": "I", "frequency": 52},
]

print(f"Data: {len(alphabet)} items")
print(alphabet[:3])


Data: 9 items
[{'letter': 'A', 'frequency': 28}, {'letter': 'B', 'frequency': 55}, {'letter': 'C', 'frequency': 43}]


In [ ]:
# Create and display bar chart
# Note: ES module loading is not yet fully supported, so this will show a helpful error
from gofish_python import chart, spread, rect, display_jupyter

try:
    node = chart(alphabet).flow(spread("letter", dir="x")).mark(rect(h="frequency"))
    display_jupyter(node, w=500, h=300, axes=True)
    print("✓ Chart created successfully")
except RuntimeError as e:
    print("⚠ Chart creation failed (expected):")
    print(str(e))
    print("\nThis is a known limitation:")
    print("  • The GoFish bundle uses ES module syntax with external dependencies")
    print("  • PythonMonkey doesn't fully support ES modules with external imports")
    print("  • The Python API structure works, but JavaScript execution needs a solution")
    print("\nPossible solutions:")
    print("  1. Build GoFish as a UMD/CommonJS bundle with dependencies bundled")
    print("  2. Use a JavaScript runtime that supports ES modules better")
    print("  3. Load solid-js separately before loading GoFish")
except Exception as e:
    print(f"⚠ Unexpected error: {e}")
    import traceback
    traceback.print_exc()


⚠ Chart creation failed: 'NoneType' object is not callable

This is expected if:
  1. JavaScript bridge isn't installed
  2. GoFish JS bundle isn't available
  3. Rendering functionality isn't fully implemented yet


Traceback (most recent call last):
  File "/var/folders/b4/8rd3hr7j13vdx2xvgxs4rwjm0000gn/T/ipykernel_67163/3073372981.py", line 6, in <module>
    node = chart(alphabet).flow(spread("letter", dir="x")).mark(rect(h="frequency"))
  File "/Users/jmp/gofish-20250316/packages/gofish-python/gofish_python/wrapper.py", line 133, in chart
    return ChartBuilder(data, options)
  File "/Users/jmp/gofish-20250316/packages/gofish-python/gofish_python/wrapper.py", line 42, in __init__
    self._js_builder = self._gofish.chart(js_data, js_options)
TypeError: 'NoneType' object is not callable


## Example 2: Stacked Bar Chart

A stacked bar chart showing fish counts by lake and species.


In [ ]:
# Sample seafood data
seafood = [
    {"lake": "Huron", "species": "Cisco", "count": 100},
    {"lake": "Huron", "species": "Trout", "count": 50},
    {"lake": "Superior", "species": "Cisco", "count": 80},
    {"lake": "Superior", "species": "Trout", "count": 120},
    {"lake": "Michigan", "species": "Cisco", "count": 90},
    {"lake": "Michigan", "species": "Trout", "count": 70},
]

print(f"Data: {len(seafood)} items")
print(seafood)


In [ ]:
# Create and display stacked bar chart
from gofish_python import chart, spread, stack, rect, display_jupyter

try:
    node = chart(seafood).flow(
        spread("lake", dir="x"),
        stack("species", dir="y")
    ).mark(rect(h="count", fill="species"))
    display_jupyter(node, w=600, h=400, axes=True)
    print("✓ Stacked chart created successfully")
except Exception as e:
    print(f"⚠ Chart creation failed: {e}")
    print("\nThis is expected if:")
    print("  1. JavaScript bridge isn't installed")
    print("  2. GoFish JS bundle isn't available")
    print("  3. Rendering functionality isn't fully implemented yet")
    import traceback
    traceback.print_exc()


## Example 3: Testing Data Conversion

Test the Python ↔ JavaScript data conversion utilities.


In [ ]:
from gofish_python.utils import to_js, from_js, ensure_list

# Test basic Python → JS conversion
test_data = {
    "name": "test",
    "value": 42,
    "items": [1, 2, 3]
}

js_data = to_js(test_data)
print(f"Python dict: {test_data}")
print(f"Converted to JS: {js_data}")
print(f"Type: {type(js_data)}")

# Test list conversion
py_list = [{"a": 1}, {"b": 2}]
js_list = to_js(py_list)
print(f"\nPython list: {py_list}")
print(f"Converted to JS: {js_list}")

# Test ensure_list
single_dict = {"x": 1, "y": 2}
list_result = ensure_list(single_dict)
print(f"\nSingle dict: {single_dict}")
print(f"Ensured list: {list_result}")


## Example 4: Testing Operators and Marks

Test that operators and marks can be created.


In [ ]:
from gofish_python.operators import spread, stack, group
from gofish_python.marks import rect, circle

# Test operator creation
try:
    spread_op = spread("letter", dir="x")
    print(f"✓ spread() operator created: {type(spread_op)}")
    
    stack_op = stack("species", dir="y")
    print(f"✓ stack() operator created: {type(stack_op)}")
    
    group_op = group("lake")
    print(f"✓ group() operator created: {type(group_op)}")
    
    # Test mark creation
    rect_mark = rect(h="frequency", fill="blue")
    print(f"✓ rect() mark created: {type(rect_mark)}")
    
    circle_mark = circle(r=5, fill="red")
    print(f"✓ circle() mark created: {type(circle_mark)}")
    
except Exception as e:
    print(f"⚠ Creation failed: {e}")
    import traceback
    traceback.print_exc()


## Troubleshooting

### Common Issues

1. **ImportError**: Package not installed
   - Solution: `cd packages/gofish-python && uv sync`

2. **JavaScript bridge not found**
   - Solution: `uv pip install pythonmonkey` or `uv pip install jsbridge`

3. **GoFish JS module not found**
   - Solution: Build GoFish JS bundle:
     ```bash
     cd packages/gofish-graphics
     pnpm install
     pnpm build
     ```
   - Or set `GOFISH_JS_PATH` environment variable

4. **Rendering not working**
   - This is expected if rendering isn't fully implemented yet
   - The API structure is in place, but rendering requires full JS bridge integration


In [ ]:
# Print package version and info
try:
    import gofish_python
    print(f"Package: gofish-graphics")
    print(f"Version: {getattr(gofish_python, '__version__', 'unknown')}")
    if hasattr(gofish_python, '__all__'):
        print(f"\nAvailable exports:")
        exports = gofish_python.__all__[:10]
        print(f"  {', '.join(exports)}...")
except Exception as e:
    print(f"Error getting package info: {e}")
